In [57]:
import numpy as np
import plotly.express as px
import polars as pl
import sys
sys.path.append('../../python')
import pdstools
settingsShowTSSpread=pl.DataFrame({"n": 5, "positives": range(1, 1001)}).with_columns(
    p=[0.01, 0.05, 0.1]
).explode("p").with_columns(evidence=pl.col("positives") / pl.col("p"))
def betaDistribution(structcol):
    return structcol.apply(
        lambda x: np.random.beta(
            x["p"] * x["evidence"], (1 - x["p"]) * x["evidence"], x["n"]
        ).tolist()
    )


thompsonSamplingSimulation = settingsShowTSSpread.with_columns(
    sampled_propensity=betaDistribution(pl.struct(["n", "p", "evidence"]))
).explode('sampled_propensity').with_columns(positives = pl.col('evidence')*pl.col('p'))

In [133]:
px.scatter(
    thompsonSamplingSimulation.to_pandas(),
    x="positives",
    y="sampled_propensity",
    color="p",
    size_max=0.3,
    opacity=0.6,
    labels={
        "sampled_propensity": "Sampled Propensity",
        "positives": "Number of positive responses in the Adaptive Model",
        "p":"Propensity"
    },
    range_y=[0, 0.2],
    title='Thompson Sampling',
    template="pega",
).update_coloraxes(showscale=False)

In [59]:
thompsonSamplingSimulation = thompsonSamplingSimulation.hstack(thompsonSamplingSimulation['positives'].cut(bins=np.array(range(int(s.min()), int(s.max())+20, 20))-1).select(bin='category'))

In [87]:
s = thompsonSamplingSimulation.groupby("p", "bin").agg(
    n=pl.count(),
    n90=(((pl.col("sampled_propensity") - pl.col("p")) / pl.col("p")) < 0.1).sum(),
    positives=pl.min("positives"),
).with_columns(pct=pl.col('n90')/pl.col('n')).sort('p', 'bin').with_columns(pl.col('p').cast(pl.Utf8).cast(pl.Categorical))


In [130]:
px.line(s.to_pandas(), x='positives', y='pct', color='p', template='none', line_group='p', title='Percentage of Sampled Propensities <br><sup>that are within 10% of the Model Propensities</sup>', labels={'pct':'Percentage', 'positives':'Percentage of positive responses in the Adaptive Model', 'p':'Propensity'})